In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option('display.max_columns', 999)

In [2]:
# load ratings.csv in /content and create dataframe with name ratings_

ratings_ = pd.read_csv('/content/ratings.csv')

In [3]:
ratings_.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


In [5]:
# load movies.csv in /content and create dataframe with name movies_

movies_ = pd.read_csv('/content/movies.csv')
movies_.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [6]:
movies_.describe()


,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.000000,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2401.000000,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,1386.651002,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1200.500000,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,2401.000000,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,3601.500000,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,4802.000000,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [7]:
movies_ = movies_.rename(columns={'id': 'movieId'})

In [8]:
# merge ratings - movies

df = pd.merge(ratings_, movies_, on='movieId', how='left')
df.head()

,userId,movieId,rating,timestamp,index,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,1,1,4.0,964982703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,3,4.0,964981247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,6,4.0,964982224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,47,5.0,964983815,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,50,5.0,964982931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# create a new df where first column is title and second column is the total users' average ratings

# Group by movie title and calculate the mean rating
mean_ratings = df.groupby('title')['rating'].mean().reset_index()

# Create a new DataFrame with title and average rating
new_df = mean_ratings[['title', 'rating']]

# Rename the rating column for clarity
mean_ratings = new_df.rename(columns={'rating': 'average_rating'})

# Display the new DataFrame
print(mean_ratings.head())

                        title  average_rating
0  10 Things I Hate About You        3.428571
1                12 Angry Men        4.500000
2                        1408        2.000000
3                  15 Minutes        4.000000
4                   16 Blocks        3.000000


In [10]:
mean_ratings = mean_ratings.sort_values(by='average_rating', ascending=False)
print(mean_ratings.head())

                                                 title  average_rating
88                                 Battle: Los Angeles             5.0
186                                 Death at a Funeral             5.0
447                                               Once             5.0
487                                        Quinceañera             5.0
119  Borat: Cultural Learnings of America for Make ...             5.0


In [11]:
# generate a new df from  movies_ & ratings_ and calculate user_similarity matrix

# Combine ratings_ and movies_ DataFrames
df = pd.merge(ratings_, movies_, on='movieId')

# Create a pivot table with users as rows, movies as columns, and ratings as values
movie_user_matrix = df.pivot_table(index='userId', columns='title', values='rating')

# Fill NaN values with 0 (or another appropriate value if preferred)
movie_user_matrix = movie_user_matrix.fillna(0)

# Calculate the user similarity matrix using cosine similarity
matrix_user_similarity = cosine_similarity(movie_user_matrix)

# Convert the similarity matrix to a DataFrame for better readability
matrix_user_similarity = pd.DataFrame(matrix_user_similarity, index=movie_user_matrix.index, columns=movie_user_matrix.index)

print(matrix_user_similarity.head())

userId       1         3         4         5         6         7         8    \
userId                                                                         
1       1.000000  0.119603  0.241664  0.143617  0.128500  0.150697  0.218894   
3       0.119603  1.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4       0.241664  0.000000  1.000000  0.165265  0.123991  0.156343  0.068496   
5       0.143617  0.000000  0.165265  1.000000  0.310811  0.231955  0.403361   
6       0.128500  0.000000  0.123991  0.310811  1.000000  0.174895  0.432614   

userId       9         10        11        12        13        14        15   \
userId                                                                         
1       0.041193  0.016903  0.149262  0.043587  0.000000  0.167650  0.108530   
3       0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.085004   
4       0.000000  0.099667  0.073341  0.122841  0.000000  0.076306  0.106655   
5       0.000000  0.186577  0.074095  0

In [12]:
matrix_user_similarity.describe().T

,count,mean,std,min,25%,50%,75%,max
userId,,,,,,,,
1,598.0,0.162039,0.103412,0.0,0.093572,0.156954,0.224797,1.0
3,598.0,0.019046,0.056541,0.0,0.000000,0.000000,0.009588,1.0
4,598.0,0.121596,0.087864,0.0,0.058026,0.118541,0.173308,1.0
5,598.0,0.164665,0.138898,0.0,0.030148,0.157837,0.254966,1.0
6,598.0,0.171088,0.140274,0.0,0.066799,0.140609,0.250385,1.0
...,...,...,...,...,...,...,...,...
606,598.0,0.147004,0.088954,0.0,0.087479,0.141359,0.196426,1.0
607,598.0,0.165416,0.117403,0.0,0.064405,0.175682,0.255637,1.0
608,598.0,0.225817,0.122755,0.0,0.142632,0.214583,0.288251,1.0


In [13]:
matrix_user_similarity.head().T

userId,1,3,4,5,6
userId,,,,,
1,1.000000,0.119603,0.241664,0.143617,0.128500
3,0.119603,1.000000,0.000000,0.000000,0.000000
4,0.241664,0.000000,1.000000,0.165265,0.123991
5,0.143617,0.000000,0.165265,1.000000,0.310811
6,0.128500,0.000000,0.123991,0.310811,1.000000
...,...,...,...,...,...
606,0.211425,0.020692,0.214916,0.166562,0.150374
607,0.306090,0.079735,0.195756,0.145161,0.183982
608,0.316625,0.057516,0.160825,0.221377,0.280108


In [14]:
#create the item_similirity_matrix

# Transpose the user-item matrix to get the item-user matrix
item_user_matrix = movie_user_matrix.T

# Calculate the item similarity matrix using cosine similarity
item_similarity_matrix = cosine_similarity(item_user_matrix)

# Convert the similarity matrix to a DataFrame for better readability
item_similarity_matrix = pd.DataFrame(item_similarity_matrix, index=item_user_matrix.index, columns=item_user_matrix.index)

print(item_similarity_matrix.head())
item_similarity_matrix.describe().T
item_similarity_matrix.head().T

title                       10 Things I Hate About You  12 Angry Men  1408  \
title                                                                        
10 Things I Hate About You                         1.0           0.0   0.0   
12 Angry Men                                       0.0           1.0   0.0   
1408                                               0.0           0.0   1.0   
15 Minutes                                         0.0           0.0   0.0   
16 Blocks                                          0.0           0.0   0.0   

title                       15 Minutes  16 Blocks  \
title                                               
10 Things I Hate About You         0.0        0.0   
12 Angry Men                       0.0        0.0   
1408                               0.0        0.0   
15 Minutes                         1.0        0.0   
16 Blocks                          0.0        1.0   

title                       20,000 Leagues Under the Sea  \
title                

title,10 Things I Hate About You,12 Angry Men,1408,15 Minutes,16 Blocks
title,,,,,
10 Things I Hate About You,1.000000,0.0,0.0,0.0,0.000000
12 Angry Men,0.000000,1.0,0.0,0.0,0.000000
1408,0.000000,0.0,1.0,0.0,0.000000
15 Minutes,0.000000,0.0,0.0,1.0,0.000000
16 Blocks,0.000000,0.0,0.0,0.0,1.000000
...,...,...,...,...,...
You Only Live Twice,0.000000,0.0,0.0,0.0,0.000000
Young Frankenstein,0.204578,0.0,0.0,0.0,0.000000
Zodiac,0.000000,0.0,0.0,0.0,0.371391


In [ ]:
# Function: given a movie title returns a list with the 10 most similar movies

def recomendar_peliculas_similares(titulo_pelicula):
  """
  Esta función toma el título de una película y devuelve una lista
  de las 10 películas más similares basándose en la similitud del coseno.

  Args:
    titulo_pelicula: El título de la película para la cual se quieren encontrar películas similares.

  Returns:
    Una lista de los títulos de las 10 películas más similares.
  """
  try:
    # Obtener la fila de similitud para la película dada
    similitudes_pelicula = item_similarity_matrix[titulo_pelicula]

    # Ordenar las similitudes de forma descendente
    similitudes_ordenadas = similitudes_pelicula.sort_values(ascending=False)

    # Excluir la propia película de la lista
    peliculas_similares = similitudes_ordenadas[1:]

    # Devolver los títulos de las 10 películas más similares
    return peliculas_similares.head(10).index.tolist()

  except KeyError:
    return f"La película '{titulo_pelicula}' no se encontró en la base de datos."


In [ ]:
titulo = "Toy Story (1995)"
recomendaciones = recomendar_peliculas_similares(titulo)
print(f"Las 10 películas más similares a '{titulo}' son:")
for peli in recomendaciones:
   print(peli)

Las 10 películas más similares a 'Toy Story (1995)' son:
L
a
 
p
e
l
í
c
u
l
a
 
'
T
o
y
 
S
t
o
r
y
 
(
1
9
9
5
)
'
 
n
o
 
s
e
 
e
n
c
o
n
t
r
ó
 
e
n
 
l
a
 
b
a
s
e
 
d
e
 
d
a
t
o
s
.


In [15]:
# prompt: corrige lo anterior para mostrar el texto en un formato correcto, horizontal

print(matrix_user_similarity.head().to_string())
print(matrix_user_similarity.describe().T.to_string())
print(matrix_user_similarity.head().T.to_string())

print(item_similarity_matrix.head().to_string())
print(item_similarity_matrix.describe().T.to_string())
print(item_similarity_matrix.head().T.to_string())

def recomendar_peliculas_similares(titulo_pelicula):
  """
  Esta función toma el título de una película y devuelve una lista
  de las 10 películas más similares basándose en la similitud del coseno.

  Args:
    titulo_pelicula: El título de la película para la cual se quieren encontrar películas similares.

  Returns:
    Una lista de los títulos de las 10 películas más similares.
  """
  try:
    # Obtener la fila de similitud para la película dada
    similitudes_pelicula = item_similarity_matrix[titulo_pelicula]

    # Ordenar las similitudes de forma descendente
    similitudes_ordenadas = similitudes_pelicula.sort_values(ascending=False)

    # Excluir la propia película de la lista
    peliculas_similares = similitudes_ordenadas[1:]

    # Devolver los títulos de las 10 películas más similares
    return peliculas_similares.head(10).index.tolist()

  except KeyError:
    return f"La película '{titulo_pelicula}' no se encontró en la base de datos."

userId       1         3         4         5         6         7         8         9         10        11        12        13        14        15        16        17        18        19        20        21        22        23        24        25        26        27        28        29        31        32        33        34        35        36        37        38        39        40        41        42        43        44        45        46        47        48        50        51        52   53        54        55        56        57        58        59      60        61        62        63        64        65        66        67        68        69        70        71        72        73        74        75        76   77        78        79        80        81        82        83        84        85        86        87        88        89        90        91        92        93        94        95        96        97        98        99        100       101       102       103      

In [19]:
title = "Zodiac"
recomendaciones = recomendar_peliculas_similares(titulo)
print(f"List of the 10 more similar movies to '{title}':")
for peli in recomendaciones:
   print(peli)

List of the 10 more similar movies to 'Zodiac':
Spirited Away
The Wedding Date
The Bourne Ultimatum
16 Blocks
Three Kings
RoboCop 3
In the Valley of Elah
Miss Congeniality
Never Back Down
Starship Troopers
